In [1]:
import pandas as pd
import numpy as np
import re
from pathlib import Path
from rapidfuzz import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
# -----------------------------
# 1. Setup
# -----------------------------
data_dir = Path('F:/Payments Reconciliation')
out_dir = data_dir / 'outputs_v2'
out_dir.mkdir(exist_ok=True)

payments = pd.read_excel(data_dir / 'payments_system.xlsx', parse_dates=['payment_timestamp'])
bank = pd.read_excel(data_dir / 'bank_statement.xlsx', parse_dates=['posting_date'])
refunds = pd.read_excel(data_dir / 'refunds.xlsx', parse_dates=['refund_timestamp'])


In [3]:
# -----------------------------
# 2. Text normalization utilities
# -----------------------------
def normalize_text(text):
    if pd.isna(text):
        return ''
    text = text.lower()
    text = re.sub(r'[^a-z0-9 ]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

payments['norm_narration'] = payments['narration'].apply(normalize_text)
bank['norm_narration'] = bank['narration'].apply(normalize_text)


In [4]:
# -----------------------------
# 3. Candidate generation using TF-IDF
# -----------------------------
vectorizer = TfidfVectorizer(min_df=2)
all_text = pd.concat([
payments['norm_narration'],
    bank['norm_narration']
])

vectorizer.fit(all_text)

payment_vecs = vectorizer.transform(payments['norm_narration'])
bank_vecs = vectorizer.transform(bank['norm_narration'])

similarity_matrix = cosine_similarity(payment_vecs, bank_vecs)


In [7]:
# -----------------------------
# 4. Matching logic with confidence scoring
# -----------------------------
results = []

for i, pay_row in payments.iterrows():
    # candidate shortlist
    sim_scores = similarity_matrix[i]
    top_candidates = np.argsort(sim_scores)[-10:][::-1]

    for j in top_candidates:
        bank_row = bank.iloc[j]

        # Date proximity score
        date_diff_days = abs((bank_row['posting_date'] - pay_row['payment_timestamp']).days)
        date_score = max(0, 1 - date_diff_days / 7)

        # Amount proximity score
        amt_diff = abs(bank_row['amount'] - pay_row['amount'])
        amt_score = max(0, 1 - (amt_diff / pay_row['amount']))

        # Narration similarity score
        narr_score = fuzz.token_set_ratio(
            pay_row['norm_narration'],
            bank_row['norm_narration']
        ) / 100

        # Customer inference
        cust_score = 1 if str(pay_row['customer_id']) in bank_row['norm_narration'] else 0

        # Weighted confidence score
        confidence = (
            0.4 * narr_score +
            0.3 * amt_score +
            0.2 * date_score +
            0.1 * cust_score
        ) * 100

        results.append({
            'payment_ref': pay_row['payment_ref'],
            'bank_txn_ref': bank_row['bank_txn_ref'],
            'confidence_score': round(confidence, 2),
            'narration_score': round(narr_score * 100, 2),
            'amount_score': round(amt_score * 100, 2),
            'date_score': round(date_score * 100, 2),
            'customer_score': cust_score * 100
        })

matches = pd.DataFrame(results)

In [8]:
# -----------------------------
# 5. Select best match per payment
# -----------------------------
best_matches = matches.sort_values('confidence_score', ascending=False)
best_matches = best_matches.groupby('payment_ref').head(1).reset_index(drop=True)

In [9]:
# -----------------------------
# 6. Classification for audit
# -----------------------------
def classify(score):
    if score >= 85:
        return 'Auto-Reconciled'
    elif score >= 70:
        return 'Review Recommended'
    elif score >= 50:
        return 'Weak Match'
    else:
        return 'Unmatched'

best_matches['recon_status'] = best_matches['confidence_score'].apply(classify)


In [10]:
# -----------------------------
# 7. Audit-friendly outputs
# -----------------------------
best_matches.to_excel(out_dir / 'reconciliation_results_detailed.xlsx', index=False)

summary = best_matches.groupby('recon_status').agg(
    transaction_count=('payment_ref', 'count'),
    avg_confidence=('confidence_score', 'mean')
).reset_index()

summary.to_excel(out_dir / 'reconciliation_summary.xlsx', index=False)

best_matches

,payment_ref,bank_txn_ref,confidence_score,narration_score,amount_score,date_score,customer_score,recon_status
0,PS-24fb6058bd,BK-95928b1acf,99.97,100.00,99.89,100.00,100,Auto-Reconciled
1,PS-86ce1ee74c,BK-414db16adc,99.97,100.00,99.90,100.00,100,Auto-Reconciled
2,PS-c13ffc694a,BK-7a46661cba,99.97,100.00,99.89,100.00,100,Auto-Reconciled
3,PS-a703f113ff,BK-2bc93b7e97,99.97,100.00,99.90,100.00,100,Auto-Reconciled
4,PS-d5666ae24f,BK-426aa3756e,99.97,100.00,99.89,100.00,100,Auto-Reconciled
...,...,...,...,...,...,...,...,...
7995,PS-b4cbf1bb40,BK-8ffe2a3deb,34.29,85.71,0.00,0.00,0,Unmatched
7996,PS-9c681eaa01,BK-6f0d8899b8,32.45,48.65,43.29,0.00,0,Unmatched
7997,PS-38cf9eaddf,BK-6f0d8899b8,22.37,48.65,9.70,0.00,0,Unmatched
7998,PS-03644ab2ba,BK-4f528efaa8,22.32,48.65,0.00,14.29,0,Unmatched
